## 전체 기간에 대한 최종 xlsx를 만든다.

In [12]:
import os
import json
import pandas as pd
from tqdm import tqdm

## read song_infos

In [13]:
song_info_file = "song_infos.jsonl"
with open(song_info_file, "r", encoding='utf-8') as f:
    lines = f.readlines()
    
song_infos = [json.loads(_) for _ in lines]
song_infos = {_["song_id"]:_ for _ in song_infos }
song_infos["34808761"]

{'song_id': '34808761',
 'song_name': '너를 위한 단어 (It’s Yours)',
 'artist_name': 'NCT DREAM',
 '앨범': 'Glitch Mode - The 2nd Album',
 'album_num': '10901349',
 '발매일': '2022.03.28',
 '장르': 'R&B/Soul',
 'lyric': "Girl 널 닮은 단어가 참 많아\n내 하루 속에\n몇 번씩 스치듯 널 만나\n어제 읽은 책\n또 좀 전에 같이 본 Romance 영화\nMy favorite song 모두 네가 있어\n \n넌 내게 멋진 여왕이 돼\n어둠에 빠진 나를 구해\n가끔씩은 슬픈 영웅이 돼\n빛이 나 oh\n밤 하늘 아래 별들 같아\n푸른 숲속의 향기 같아\n다 너의 수식어 yeah yeah\n너로 가득 차 있는\n나의 세상 속 oh oh\n난 지금 노래 가사로\n너를 얘기하고 oh oh\n머릿속에 춤추는 멋진 말\nIt's yours It's yours It's yours\n(yeah It's yours 널 위한 것)\n \nSay yeah yeah yeah yeah\nCause you’re my word\n그 어떤 말이 너만큼 많이 빛날까\nIt's yours It's yours It's yours\n모두 다 주고 싶어\nGirl 유치한 생각이 넘쳐나\n주인공은 You \n난 온갖 시를 써 Up all night\noh 눈 감고 누우면\n머릿속에 네가 떠다니고\n내일 무슨 말로\n웃게 할지 고민만 해\n알아 솔직히 It’s crazy oh\n남기고 싶어 모든 곳에\n어떤 말보다 아름답게\n보고 계속 봐도 보고 싶게\n널 말이야 ooh\n뭘로 써봐도 난 부족해\n모든 언어로 남겨둘래\n다 너의 수식어 yeah yeah\n너로 가득 차 있는\n나의 세상 속 oh oh\n난 지금 노래 가사로\n너를 얘기하고 oh oh\n머릿속에 춤추는 멋진 말\nIt's yours It's yours It's yours\n(yeah I

## read artist_info file

In [14]:
artist_info_file = "artist_infos.jsonl"
with open(artist_info_file, "r", encoding='utf-8') as f:
    lines = f.readlines()
    artist_infos = [json.loads(_) for _ in lines]
    
artist_infos = {_["artist_num"]: _ for _ in artist_infos}
artist_infos["2636017"]

{'artist_num': '2636017',
 '이름': 'PRODUCE X 101',
 '활동년대': '2010 년대',
 '유형': '그룹|남성',
 '장르': '댄스, 발라드',
 '소속사명': 'CJ E&M'}

## read chart data

In [15]:
parsed_chart_path = "parsed_chart_html"
jsonl_files = os.listdir(parsed_chart_path)
jsonl_files = [_ for _ in sorted(jsonl_files) if _.endswith("jsonl")]

In [16]:
chart_data = []
for jsonl_file in jsonl_files:
    with open(os.path.join(parsed_chart_path, jsonl_file), "r", encoding='utf-8') as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
    chart_data.extend(docs)

In [17]:
len(chart_data)

98989

In [18]:
chart_data[4555]

{'ranking': '11',
 'title_num': '985474',
 'title_name': '그래도 살아가겠지',
 'album_num': '309078',
 'album_name': 'The Very First',
 'artists': [['174997', '가비엔제이']],
 'yearweek': '200547',
 'period': '20051121-20051127'}

## merge table

In [19]:
lasts = []
for line in tqdm(chart_data):
    num_artists = len(line["artists"])
    if num_artists > 1:
        artist_activity_type = "프로젝트"
        artist_id = "|".join([_[0] for _ in line["artists"]])
        artist_name = "|".join([_[1] for _ in line["artists"]])
    elif num_artists == 0:
        artist_activity_type = ""
        artist_id = ""
        artist_name = ""
    else:
        artist_id = line["artists"][0][0]
        artist_name = line["artists"][0][1]
        artist_activity_type = artist_infos.get(artist_id, {}).get("유형", "")

    song_id = line["title_num"]
    song_genre = song_infos.get(song_id, {}).get("장르", "")

    last = {}
    last["yearweek"] = line["yearweek"]
    last["period"] = line["period"]
    last["ranking"] = line["ranking"]
    last["song_id"] = line["title_num"]
    last["song_name"] = line["title_name"]
    last["song_genre"] = song_genre
    
    last["artist_id"] = artist_id
    last["artist_name"] = artist_name
    last["artist_activity_type"] = artist_activity_type
    
    last["album_id"] = line["album_num"]
    last["album_name"] = line["album_name"]
        
    lasts.append(last)


100%|██████████| 98989/98989 [00:00<00:00, 258400.85it/s]


In [20]:
lasts[-100]

{'yearweek': '202352',
 'period': '20231225-20231231',
 'ranking': '1',
 'song_id': '36910957',
 'song_name': 'Perfect Night',
 'song_genre': '',
 'artist_id': '3092950',
 'artist_name': 'LE SSERAFIM (르세라핌)',
 'artist_activity_type': '그룹|여성',
 'album_id': '11352904',
 'album_name': 'Perfect Night'}

## make pandas df

In [21]:
df = pd.DataFrame(lasts)

In [22]:
df

,yearweek,period,ranking,song_id,song_name,song_genre,artist_id,artist_name,artist_activity_type,album_id,album_name
0,200501,20050103-20050109,1,623250,보통날,발라드,100007,god,그룹|남성,43691,보통날
1,200501,20050103-20050109,2,631232,둘이서 (CF - 빙그레 '바나나맛 우유'),댄스,41894,채연,솔로|여성,44462,Virginalness Bloom
2,200501,20050103-20050109,3,642420,겨울일기,발라드,10575,장나라,솔로|여성,45305,나의 이야기
3,200501,20050103-20050109,4,623157,빙고,댄스,100022,코요태,그룹|혼성,43648,Rainbow 7집
4,200501,20050103-20050109,5,622912,헤어지자고 (MBC 슬픈연가 삽입곡),발라드,14801,윤건,솔로|남성,43532,Now Is.. 2집
...,...,...,...,...,...,...,...,...,...,...,...
98984,202352,20231225-20231231,96,418168,희재,"발라드, 국내영화",3305,성시경,솔로|남성,28985,국화꽃 향기 OST
98985,202352,20231225-20231231,97,36105548,Teddy Bear,댄스,2899290,STAYC(스테이씨),그룹|여성,11165170,Teddy Bear
98986,202352,20231225-20231231,98,37065199,Stay With Me (Feat. Skinny Brown),,2112495,ASH ISLAND,솔로|남성,11385652,Stay With Me
98987,202352,20231225-20231231,99,31927275,한 페이지가 될 수 있게,록/메탈,894864,DAY6 (데이식스),그룹|남성,10307346,The Book of Us : Gravity


In [23]:
df.to_excel("melon_weekly_top100_200501-202352.xlsx", index=False, engine="openpyxl")